# Enfrentando o Dragão
**Mural de Quests 04 - Jardim do Palácio**

**Disciplina:** Aprendizado de Máquina

**Professor:** Daniel R. Cassar

**Alunas:** Lília Gavazza, Lívia Aragão, Sophia Alves

Esse é o primeiro teste dos cinco modelos a serem aplicados no dataset escolhido.

In [1]:
# Importando bibliotecas e funções necessárias
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from optuna import create_study
from sklearn.model_selection import cross_val_score

# [Modelos]
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

C:\Users\livia25013\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Carregando dados

In [17]:
colunas = ["Crystal System", "Sites", "Energy Above Hull", 
           "Formation Energy", "Volume", "Density", "Band Gap", "Is Gap Direct", "Is Metal",
           "Magnetic Ordering"]

nome_do_arquivo = "magnetismo_Co(in).csv"
magnetic_co = pd.read_csv(nome_do_arquivo)

magnetic_co = magnetic_co.dropna() # retirar linhas com valores vazios 
magnetic_co = magnetic_co.reindex(colunas, axis = 1)

TAMANHO_TESTE = 0.1 
SEMENTE_ALEATORIA = 37

indices = magnetic_co.index
indices_treino, indices_teste = train_test_split(
    indices, test_size = TAMANHO_TESTE, random_state = SEMENTE_ALEATORIA)

# Definição dos DataFrames com os dados de treino e de teste
treino_mco = magnetic_co.loc[indices_treino]
teste_mco = magnetic_co.loc[indices_teste]

FEATURES = ["Sites", "Energy Above Hull", 
           "Formation Energy", "Volume", "Density", "Band Gap"]
TARGET = ["Magnetic Ordering"]

# Separação e seleção dos valores de features e target por DF
X_treino = treino_mco.reindex(FEATURES, axis = 1).values
y_treino = treino_mco.reindex(TARGET, axis = 1).values.ravel() # deixa os dados em 1D - maior precisão

X_teste = teste_mco.reindex(FEATURES, axis = 1).values
y_teste = teste_mco.reindex(TARGET, axis = 1).values.ravel() # deixa os dados em 1D - maior precisão

In [18]:
print(X_treino)

[[ 1.00000000e+01  0.00000000e+00 -1.77834970e-02  1.93739246e+02
   7.97589115e+00  0.00000000e+00]
 [ 3.00000000e+00  1.63028660e-01 -9.09404000e-01  7.84801123e+01
   1.92400502e+00  0.00000000e+00]
 [ 4.80000000e+01  3.37875880e-02 -2.92326920e-02  6.88194785e+02
   9.03159953e+00  0.00000000e+00]
 ...
 [ 6.00000000e+00  2.21607677e-01  1.57687893e-01  8.27475122e+01
   9.49501266e+00  0.00000000e+00]
 [ 3.00000000e+00  1.92061920e-02 -1.65673970e-01  6.88033664e+01
   7.58146686e+00  0.00000000e+00]
 [ 3.80000000e+01  0.00000000e+00 -8.69745460e-02  4.57828231e+02
   9.71839828e+00  0.00000000e+00]]


#### Baseline 
Esse é o modelo base, ele apenas prevê a classe mais frequente para todos os dados. Não existe necessidade, portanto de avaliar o melhor conjunto de hiperparâmetro, pois eles não existem nesse caso.

In [19]:
pipeline_bsl = make_pipeline(
    StandardScaler(),
    DummyClassifier(strategy='stratified', random_state=SEMENTE_ALEATORIA)
)
    
pipeline_bsl.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto1 = pipeline_bsl.predict(X_teste)

acuracia_bsl = accuracy_score(y_verdadeiro, y_previsto1)
print(f"Acurácia - Baseline: {round((acuracia_bsl)*100, 2)}%")

Acurácia - Baseline: 48.15%


Esse desempenho será o nosso norte quanto a real capacidade de generalização dos próximos algoritmos, pensando que quanto maior e mais distante dessa base, melhor é o modelo induzido.

### Otimização de Hiperparâmetros com `optuna`
O `optuna` é um módulo de python que permite uma otimização de hiperparâmetros por uma espécie de "conjunto de tentativas". O `trial` é um objeto dele que sorteia e sugere parâmetros para o modelo, criando seu espaço de busca de hiperparâmetros. 

Disso, é definida a função objetivo para medir, reforçada por uma validação cruzada k-fold, a métrica sob análise, nesse caso, a acurácia. Efetivamente o que ocorre é que o conjunto de hiperparâmetros sugerido no passo anterior é avaliado por essa estratégia, que devolve a estimativa de desempenho dele para guiar e comparar com as próximas escolhas. Isso é feito por uma subdivisão dos dados de treino em $k$ subconjuntos, que serão utilizados como dados de teste uma vez. A média das $k$ estimativas de desempenho obtidas é mais robusta em determinar a performance do modelo "induzido".

Por fim, a criação do objeto de estudo define o otimizador desse problema de maximização que será rodado com módulo `optimize` um determinado número de tentativas (100). Os melhores parâmetros encontrados serão os utilizados para realmente induzir o modelo e calcular sua acurácia

#### k-NN Classificador
Esse modelo se baseia na moda de classes de n dos seus vizinhos para prever um dado desconhecido. Seus principais hiperparâmetros são:
- `n_neighbors`: o número de vizinhos a serem considerados na análise da moda;
- `weights`: como as classes dos vizinhos influenciam na previsão, de forma uniforme com todos tendo o meso peso ou de forma guiada pela distância, os mais próximos tem maior peso;
- `p`: o tipo de distância considerada na determinação dos vizinhos mais próximos.

Também será analisada a influência/necessidade da normalização dos dados.

In [20]:
def instancia_kNN(trial):
    parametros_kNN = {
        "n_neighbors": trial.suggest_int("num_vizinhos", 1, 100),
        "weights": trial.suggest_categorical("pesos", ["uniform", "distance"]),
        "p": trial.suggest_float("potencia_minkowski", 1, 2),
        "n_jobs": -1 # processamento
    }
    
    modelo_kNN = KNeighborsClassifier(**parametros_kNN)

    return modelo_kNN           

def func_obj_kNN(trial, X, y, n_folds):
    modelo = instancia_kNN(trial)

    metricas_kNN = cross_val_score(
        modelo,
        X,
        y,
        scoring="f1_weighted",
        cv=n_folds,
        error_score='raise'
    )

    return metricas_kNN.mean()

NOME_DO_ESTUDO = "kNN"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-02 20:23:06,819] Using an existing study with name 'kNN' instead of creating a new one.


In [21]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_kNN(trial):
    return func_obj_kNN(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_kNN, n_trials=NUM_TENTATIVAS)

[I 2025-11-02 20:23:08,232] Trial 35 finished with value: 0.4681163345580921 and parameters: {'num_vizinhos': 8, 'pesos': 'uniform', 'potencia_minkowski': 1.433921898655268}. Best is trial 22 with value: 0.5322570075527107.
[I 2025-11-02 20:23:08,478] Trial 36 finished with value: 0.5041660137429947 and parameters: {'num_vizinhos': 15, 'pesos': 'distance', 'potencia_minkowski': 1.836095208777642}. Best is trial 22 with value: 0.5322570075527107.
[I 2025-11-02 20:23:08,723] Trial 37 finished with value: 0.4300004423646901 and parameters: {'num_vizinhos': 39, 'pesos': 'uniform', 'potencia_minkowski': 1.2669653520410398}. Best is trial 22 with value: 0.5322570075527107.
[I 2025-11-02 20:23:12,943] Trial 38 finished with value: 0.5124652165667472 and parameters: {'num_vizinhos': 99, 'pesos': 'distance', 'potencia_minkowski': 1.0799164752257537}. Best is trial 22 with value: 0.5322570075527107.
[I 2025-11-02 20:23:13,165] Trial 39 finished with value: 0.504274027859908 and parameters: {'num

In [22]:
melhor_trial_kNN = objeto_de_estudo.best_trial

modelo_kNN = instancia_kNN(melhor_trial_kNN)
modelo_kNN.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto2 = modelo_kNN.predict(X_teste)

f1_kNN = f1_score(y_verdadeiro, y_previsto2, average = "weighted")

print(f"Melhor trial: {melhor_trial_kNN.number} - Parâmetros: {melhor_trial_kNN.params}")
print(f"Acurácia - kNN: {f1_kNN}")

Melhor trial: 42 - Parâmetros: {'num_vizinhos': 1, 'pesos': 'distance', 'potencia_minkowski': 1.4555721357211413}
Acurácia - kNN: 0.527065527065527


#### Árvore de Decisão
As previsões são feitas por meio da sudivisão dos dados de acordo com limiares de um determinado critério. Seus principais hiperparâmetros são:
- `max_leaf_nodes`: número máximo de vértices folha;
- `max_depth`: máxima profundidade da árvore de decisão gerada;
- `min_samples_split`: número mínimo de exemplos para gerar uma divisão condicional deles;
- `min_samples_leaf`: número mínimo de exemplos em um vértice folha.

Novamente, a possibilidade de normalização foi considerada.

In [23]:
def instancia_ad(trial):
    parametros_ad = {
        "max_leaf_nodes": trial.suggest_int("num_maximo_nos", 2, 20),
        "max_depth": trial.suggest_categorical("profundidade_maxima", [None, 3, 6, 9, 12, 18, 21, 24]),
        "min_samples_split": trial.suggest_int("min_exemplos_vcondicional", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_exemplos_vfolha", 1, 20),
        "random_state": 73 #reprotibilidade
    }

    normalizar = trial.suggest_categorical("normalizacao", [False, True])

    if normalizar == True:
        modelo_ad = make_pipeline(
            StandardScaler(),
            DecisionTreeClassifier(**parametros_ad)
        )

    else:
        modelo_ad = DecisionTreeClassifier(**parametros_ad)

    return modelo_ad           

def func_obj_ad(trial, X, y, n_folds):
    modelo = instancia_ad(trial)

    metricas_ad = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        
        cv=n_folds
    )

    return metricas_ad.mean()

NOME_DO_ESTUDO = "arvore_de_decisao"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-02 20:23:49,417] A new study created in RDB with name: arvore_de_decisao


In [24]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_ad(trial):
    return func_obj_ad(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_ad, n_trials=NUM_TENTATIVAS)

[I 2025-11-02 20:23:50,581] Trial 0 finished with value: 0.5034013605442177 and parameters: {'num_maximo_nos': 19, 'profundidade_maxima': None, 'min_exemplos_vcondicional': 18, 'min_exemplos_vfolha': 20, 'normalizacao': True}. Best is trial 0 with value: 0.5034013605442177.
[I 2025-11-02 20:23:50,734] Trial 1 finished with value: 0.491156462585034 and parameters: {'num_maximo_nos': 16, 'profundidade_maxima': 21, 'min_exemplos_vcondicional': 8, 'min_exemplos_vfolha': 17, 'normalizacao': True}. Best is trial 0 with value: 0.5034013605442177.
[I 2025-11-02 20:23:51,505] Trial 2 finished with value: 0.536734693877551 and parameters: {'num_maximo_nos': 17, 'profundidade_maxima': 18, 'min_exemplos_vcondicional': 6, 'min_exemplos_vfolha': 2, 'normalizacao': False}. Best is trial 2 with value: 0.536734693877551.
[I 2025-11-02 20:23:51,665] Trial 3 finished with value: 0.5079931972789116 and parameters: {'num_maximo_nos': 8, 'profundidade_maxima': 3, 'min_exemplos_vcondicional': 11, 'min_exempl

In [25]:
melhor_trial_ad = objeto_de_estudo.best_trial

modelo_ad = instancia_ad(melhor_trial_ad)
modelo_ad.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto3 = modelo_ad.predict(X_teste)

acuracia_ad = accuracy_score(y_verdadeiro, y_previsto3)

print(f"Melhor trial: {melhor_trial_ad.number} - Parâmetros: {melhor_trial_ad.params}")
print(f"Acurácia - Árvore de Decisão: {acuracia_ad}")

Melhor trial: 95 - Parâmetros: {'num_maximo_nos': 15, 'profundidade_maxima': 12, 'min_exemplos_vcondicional': 5, 'min_exemplos_vfolha': 5, 'normalizacao': False}
Acurácia - Árvore de Decisão: 0.6666666666666666


#### Floresta Aleatória
Nesse algoritmos são projetadas vários conjuntos de árvores de decisão, tentando criar um modelo de maior variância e menor viés. Seus principais hiperparâmetros são:
- `n_estimators`: número de árvores por comitê;
- `min_samples_split`: número mínimo de exemplos para gerar uma divisão condicional deles;
- `min_samples_leaf`: número mínimo de exemplos em um vértice folha;
- `max_features`: fração de atributos usada em cada _split_ das árvores;
- `bootstrap`: se ocorre uma amostragem do tipo _bootstrap_ dos exemplos para treinar as árvores.

In [26]:
def instancia_fa(trial):
    parametros_fa = {
        "n_estimators": trial.suggest_int("num_arvores", 50, 500, log=True),
        "min_samples_split": trial.suggest_int("min_exemplos_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_exemplos_folha", 1, 10),
        "max_features": trial.suggest_float("num_max_atributos", 0, 1),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "n_jobs": -1, #processamento
        "random_state": 73 #reprodutibilidade
    }

    normalizar = trial.suggest_categorical("normalizacao", [False, True])
    
    if normalizar == True:
        modelo_fa = make_pipeline(
            StandardScaler(),
            RandomForestClassifier(**parametros_fa)
        )

    else:
        modelo_fa= RandomForestClassifier(**parametros_fa)

    return modelo_fa

def func_obj_fa(trial, X, y, num_folds):
    modelo = instancia_fa(trial)

    metricas_fa = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_fa.mean()

NOME_DO_ESTUDO = "floresta_aleatoria"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-02 20:24:14,284] Using an existing study with name 'floresta_aleatoria' instead of creating a new one.


In [27]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_fa(trial):
    return func_obj_fa(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_fa, n_trials=NUM_TENTATIVAS)

[I 2025-11-02 20:24:16,728] Trial 100 finished with value: 0.5450680272108843 and parameters: {'num_arvores': 304, 'min_exemplos_split': 9, 'min_exemplos_folha': 6, 'num_max_atributos': 0.9580390478997421, 'bootstrap': False, 'normalizacao': True}. Best is trial 14 with value: 0.6195578231292518.
[I 2025-11-02 20:24:18,083] Trial 101 finished with value: 0.549234693877551 and parameters: {'num_arvores': 274, 'min_exemplos_split': 9, 'min_exemplos_folha': 6, 'num_max_atributos': 0.9331299714921375, 'bootstrap': False, 'normalizacao': True}. Best is trial 14 with value: 0.6195578231292518.
[I 2025-11-02 20:24:19,830] Trial 102 finished with value: 0.5450680272108843 and parameters: {'num_arvores': 358, 'min_exemplos_split': 10, 'min_exemplos_folha': 6, 'num_max_atributos': 0.8734825019993057, 'bootstrap': False, 'normalizacao': True}. Best is trial 14 with value: 0.6195578231292518.
[I 2025-11-02 20:24:23,044] Trial 103 finished with value: 0.5450680272108843 and parameters: {'num_arvore

In [28]:
melhor_trial_fa = objeto_de_estudo.best_trial

modelo_fa = instancia_fa(melhor_trial_fa)
modelo_fa.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto4 = modelo_fa.predict(X_teste)

acuracia_fa = accuracy_score(y_verdadeiro, y_previsto4)

print(f"Melhor trial: {melhor_trial_fa.number} - Parâmetros: {melhor_trial_fa.params}")
print(f"Acurácia - Floresta Aleatória: {acuracia_fa:.2f}")

Melhor trial: 14 - Parâmetros: {'num_arvores': 92, 'min_exemplos_split': 7, 'min_exemplos_folha': 3, 'num_max_atributos': 0.5816368920742446, 'bootstrap': False, 'normalizacao': True}
Acurácia - Floresta Aleatória: 0.63


#### Naive Bayes Gaussiano
Esse algoritmo assume _features_ independentes e faz previsões baseadas nas probailidades das classes assumindo as _features_ em distribuição gaussiana e o Teorema de Bayes. Seu principal hiperparâmetro é:
- `var_smoothing`: controla a variância que afeta as distribuições das _features_, para aumentar a estabilidade da previsão.

In [29]:
def instancia_nbg(trial):
    parametros_nbg = {
        "var_smoothing": trial.suggest_float("var_smoothing", 1e-11, 1e-1, log=True)
    }

    normalizar = trial.suggest_categorical("normalizacao", [False, True])
    
    if normalizar == True:
        modelo_nbg= make_pipeline(
            StandardScaler(),
            GaussianNB(**parametros_nbg)
        )

    else:
        modelo_nbg= GaussianNB(**parametros_nbg)
    
    return modelo_nbg

def func_obj_nbg(trial, X, y, num_folds):
    modelo = instancia_nbg(trial)

    metricas_nbg = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_nbg.mean()

NOME_DO_ESTUDO = "naive_bayes_gaussiano"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-02 20:26:23,712] A new study created in RDB with name: naive_bayes_gaussiano


In [30]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_nbg(trial):
    return func_obj_nbg(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_nbg, n_trials=NUM_TENTATIVAS)

[I 2025-11-02 20:26:25,145] Trial 0 finished with value: 0.4174319727891156 and parameters: {'var_smoothing': 6.3413354219271904e-09, 'normalizacao': True}. Best is trial 0 with value: 0.4174319727891156.
[I 2025-11-02 20:26:25,239] Trial 1 finished with value: 0.4505102040816326 and parameters: {'var_smoothing': 2.2820131514273987e-05, 'normalizacao': True}. Best is trial 1 with value: 0.4505102040816326.
[I 2025-11-02 20:26:25,331] Trial 2 finished with value: 0.4629251700680272 and parameters: {'var_smoothing': 5.009414728266102e-05, 'normalizacao': True}. Best is trial 2 with value: 0.4629251700680272.
[I 2025-11-02 20:26:25,430] Trial 3 finished with value: 0.4792517006802721 and parameters: {'var_smoothing': 0.00025794182429911137, 'normalizacao': True}. Best is trial 3 with value: 0.4792517006802721.
[I 2025-11-02 20:26:25,509] Trial 4 finished with value: 0.413265306122449 and parameters: {'var_smoothing': 8.790491557488308e-11, 'normalizacao': True}. Best is trial 3 with value

In [32]:
melhor_trial_nbg = objeto_de_estudo.best_trial

modelo_nbg = instancia_nbg(melhor_trial_nbg)
modelo_nbg.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto5 = modelo_nbg.predict(X_teste)

acuracia_nbg = accuracy_score(y_verdadeiro, y_previsto5)

print(f"Melhor trial: {melhor_trial_nbg.number} - Parâmetros: {melhor_trial_nbg.params}")
print(f"Acurácia - Naive Bayes Gaussiano: {acuracia_nbg:.2f}")

Melhor trial: 23 - Parâmetros: {'var_smoothing': 9.369616371257635e-06, 'normalizacao': False}
Acurácia - Naive Bayes Gaussiano: 0.63


#### Support Vector Classification - SVC
Nesse modelo, a multiclassficação é subdividida em problemas de classificação binária. Em cada um deles, procura o hiperplano que melhora separa as diferentes classes. Seus principais hiperparâmetros são:
- `kernel`: método de separação dos grupos;
- `C`: parâmetro de regularização;
- `gamma`: controla a influência de cada ponto de treino em _kernels_ não lineares;
- `max_iter`:

In [33]:
def instancia_svc(trial):
    parametros_svc = {
        "kernel": trial.suggest_categorical("kernel", ["rbf", "poly", "sigmoid"]),
        "C": trial.suggest_float("C", 0.01, 100, log=True),
        "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
        "max_iter": 5000,
        "random_state": 73 #reprodutibilidade
    }

    normalizar = trial.suggest_categorical("normalizacao", [False, True])

    if normalizar == True:
        modelo_svc = make_pipeline(
            StandardScaler(),
            SVC(**parametros_svc)
        )

    else:
        modelo_svc = SVC(**parametros_svc)

    return modelo_svc  



def func_obj_svc(trial, X, y, num_folds):
    modelo = instancia_svc(trial)

    metricas_svc = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_svc.mean()

NOME_DO_ESTUDO = "svc"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-02 20:27:53,706] A new study created in RDB with name: svc


In [34]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_svc(trial):
    return func_obj_svc(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_svc, n_trials=NUM_TENTATIVAS)

[I 2025-11-02 20:27:54,990] Trial 0 finished with value: 0.4505102040816327 and parameters: {'kernel': 'rbf', 'C': 9.60551720229225, 'gamma': 'scale', 'normalizacao': False}. Best is trial 0 with value: 0.4505102040816327.
[I 2025-11-02 20:27:55,120] Trial 1 finished with value: 0.400765306122449 and parameters: {'kernel': 'sigmoid', 'C': 0.03745264786086611, 'gamma': 'scale', 'normalizacao': True}. Best is trial 0 with value: 0.4505102040816327.
[I 2025-11-02 20:27:55,229] Trial 2 finished with value: 0.4461734693877551 and parameters: {'kernel': 'poly', 'C': 0.9740136686346373, 'gamma': 'auto', 'normalizacao': True}. Best is trial 0 with value: 0.4505102040816327.
[I 2025-11-02 20:27:55,338] Trial 3 finished with value: 0.400765306122449 and parameters: {'kernel': 'sigmoid', 'C': 81.74988686168889, 'gamma': 'auto', 'normalizacao': False}. Best is trial 0 with value: 0.4505102040816327.
[I 2025-11-02 20:27:55,462] Trial 4 finished with value: 0.400765306122449 and parameters: {'kernel

In [35]:
melhor_trial_svc = objeto_de_estudo.best_trial

modelo_svc = instancia_svc(melhor_trial_svc)
modelo_svc.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto7 = modelo_svc.predict(X_teste)

acuracia_svc = accuracy_score(y_verdadeiro, y_previsto7)

print(f"Melhor trial: {melhor_trial_svc.number} - Parâmetros: {melhor_trial_svc.params}")
print(f"Acurácia - SVC: {acuracia_svc}")

Melhor trial: 56 - Parâmetros: {'kernel': 'rbf', 'C': 3.4211441905056437, 'gamma': 'auto', 'normalizacao': True}
Acurácia - SVC: 0.5185185185185185


#### Regressão Logística

In [36]:
def instancia_lr(trial):
    parametros_lr = {
        'penalty':  trial.suggest_categorical("penalty", ['l1','l2']),
        'C': trial.suggest_float("C", 1e-4, 1e4, log=True),
        'solver': "saga",
        'max_iter': trial.suggest_int("max_iter", 100, 5000, log=True),
        'tol': 1e-2,
        'random_state': 73
}

    normalizar = trial.suggest_categorical("normalizacao", [False, True])

    if normalizar == True:
        modelo_lr = make_pipeline(
            StandardScaler(),
            LogisticRegression(**parametros_lr)
        )

    else:
        modelo_lr = LogisticRegression(**parametros_lr)

    return modelo_lr


def func_obj_lr(trial, X, y, num_folds):
    modelo = instancia_lr(trial)

    metricas_svc = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_svc.mean()

NOME_DO_ESTUDO = "lr"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-02 20:28:25,562] A new study created in RDB with name: lr


In [37]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_lr(trial):
    return func_obj_lr(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_lr, n_trials=NUM_TENTATIVAS)

[I 2025-11-02 20:28:26,546] Trial 0 finished with value: 0.565561224489796 and parameters: {'penalty': 'l1', 'C': 13.951818270961702, 'max_iter': 488, 'normalizacao': True}. Best is trial 0 with value: 0.565561224489796.
[I 2025-11-02 20:28:26,654] Trial 1 finished with value: 0.4667517006802721 and parameters: {'penalty': 'l2', 'C': 0.37023500358175515, 'max_iter': 273, 'normalizacao': False}. Best is trial 0 with value: 0.565561224489796.
[I 2025-11-02 20:28:26,791] Trial 2 finished with value: 0.4667517006802721 and parameters: {'penalty': 'l1', 'C': 87.19989680712074, 'max_iter': 4003, 'normalizacao': False}. Best is trial 0 with value: 0.565561224489796.
[I 2025-11-02 20:28:26,906] Trial 3 finished with value: 0.400765306122449 and parameters: {'penalty': 'l1', 'C': 0.006665741741221668, 'max_iter': 324, 'normalizacao': False}. Best is trial 0 with value: 0.565561224489796.
[I 2025-11-02 20:28:27,000] Trial 4 finished with value: 0.400765306122449 and parameters: {'penalty': 'l1',

In [38]:
melhor_trial_lr = objeto_de_estudo.best_trial

modelo_lr = instancia_lr(melhor_trial_lr)
modelo_lr.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto7 = modelo_lr.predict(X_teste)

acuracia_lr = accuracy_score(y_verdadeiro, y_previsto7)

print(f"Melhor trial: {melhor_trial_lr.number} - Parâmetros: {melhor_trial_lr.params}")
print(f"Acurácia - SVC: {acuracia_lr}")

Melhor trial: 10 - Parâmetros: {'penalty': 'l2', 'C': 0.1387153019925885, 'max_iter': 107, 'normalizacao': True}
Acurácia - SVC: 0.5185185185185185


### Referências
1. `sckit-learn` - Geração de Datasets. Disponível em: [https://scikit-learn.org/stable/datasets/sample_generators.html#sample-generators](https://scikit-learn.org/stable/datasets/sample_generators.html#sample-generators)
2. `sckit-learn` - Validação Cruzada. Disponível em: [https://scikit-learn.org/stable/modules/cross_validation.html](https://scikit-learn.org/stable/modules/cross_validation.html)
3. `sckit-learn` - Naive Bayes. Disponível em: [https://scikit-learn.org/stable/modules/naive_bayes.html](https://scikit-learn.org/stable/modules/naive_bayes.html)
4. BAELDUNG. Multiclass Classification Using Support Vector Machines | Baeldung on Computer Science. Disponível em: [https://www.baeldung.com/cs/svm-multiclass-classification](https://www.baeldung.com/cs/svm-multiclass-classification)
5. `sckit-learn` - `SVC`. Disponível em: [https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) 
6. CASSAR, Daniel. Materiais de Aula.